In [2]:
%matplotlib qt5
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tabulate import tabulate
import awkward as ak
import numpy as np

from python.analysis import Master, vectors

from apps.prod4a_merge_study import EventSelection, ShowerMergeQuantities


events = Master.Data("work/ROOTFiles/Prod4a_6GeV_BeamSim_00_evd.root", True)

# create hit space point arrays
events.recoParticles.spacePoints = ak.zip({"x" : events.io.Get("reco_daughter_allShower_spacePointX"), 
                                           "y" : events.io.Get("reco_daughter_allShower_spacePointY"),
                                           "z" : events.io.Get("reco_daughter_allShower_spacePointZ")})
start_showers = EventSelection(events)

mask = np.logical_or(events.recoParticles.cnnScore > 0.64, np.logical_or(*start_showers)) # select PFOs which pass CNN selection or are start showers
mask = np.logical_or(events.recoParticles.beam_number == events.recoParticles.number, mask)
events.Filter([mask])
start_showers = [start_showers[i][mask] for i in range(2)]


#* get boolean mask of PFP's to merge
to_merge = np.logical_not(np.logical_or(*start_showers))

q = ShowerMergeQuantities(events, to_merge)
q.Evaluate(events, start_showers)

#* get boolean mask of PFP's which are actual fragments of the starting showers
start_shower_ID = events.trueParticlesBT.number[np.logical_or(*start_showers)]
to_merge_ID = events.trueParticlesBT.number[to_merge]
signal = [to_merge_ID == start_shower_ID[:, i] for i in range(2)] # signal are the PFOs which is a fragment of the ith starting shower

#* define signal and background
signal_all = np.logical_or(*signal)
signal_all = signal_all[q.null]
background = np.logical_not(signal_all) # background is all other PFOs unrelated to the pi0 decay
signal = [signal[i][q.null] for i in range(2)]

/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to filters.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to filename.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to _TrueParticleData__pi0_MC.
  warnings.warn(f"Couldn't apply filters to {var}.")


'BeamMCFilter' executed in 0.8265s
number of dalitz decays: 1
'ApplyBeamFilter' executed in 0.6704s
\begin{tabular}{lllll}
\hline
 event selection                   & type        & number of events & percentage of events removed & percentage of events remaining \\
 no selection                      & -           & 749              & -                            &                                \\
 beam -\ensuremath{>} pi0 + X                   & truth       & 260              & 65.28704939919893            & 100                            \\
 pi+ beam                          & backtracked & 208              & 20.0                         & 80.0                           \\
 diphoton decay                    & truth       & 207              & 0.4807692307692308           & 79.61538461538461              \\
 beam particle                     & reco        & 128              & 38.16425120772947            & 49.23076923076923              \\
 nPFP \ensuremath{>} 1                         

/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to _RecoParticleData__beam_number.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to _RecoParticleData__beamVertex.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to run.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to subRun.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/Master.py:50: UserWarning: Couldn't apply filters to eventNum.
  warnings.warn(f"Couldn't apply filters to {var}.")


'Evaluate' executed in 13.1015s


In [223]:
nEvents = ak.num(events.recoParticles.spacePoints.x, 0)
print(f"number of events: {nEvents}")
eventNum = 85 # 28, 38, 39, 40, 62, 69, 74, 84, 85
nPFO = ak.num(events.recoParticles.spacePoints.x)
showSignal = True
showBackground = False

nSignal = ak.num(signal_all[signal_all])
print(f"number of signal PFOs: {nSignal[eventNum]}")

start_showers_merged = np.logical_or(*start_showers)
start_shower_energy = [ak.ravel(events.recoParticles.energy[i][eventNum]) for i in start_showers]
true_start_shower_energy = [ak.ravel(events.trueParticlesBT.energy[i][eventNum]) for i in start_showers]
signal_PFO_energy = [ak.ravel(events.recoParticles.energy[i][eventNum]) for i in signal]

reco_start_shower_momentum = [events.recoParticles.momentum[i][eventNum] for i in start_showers]
reco_PFO_momentum = [events.recoParticles.momentum[i][eventNum] for i in signal]
true_start_shower_momentum = [events.trueParticlesBT.momentum[i][eventNum] for i in start_showers]

table = [[f"start shower energy {i}", start_shower_energy[i]] for i in range(2)]
table.extend([[f"true shower energy {i}", true_start_shower_energy[i]] for i in range(2)])
table.extend([[f"signal PFO energy {i}", signal_PFO_energy[i]] for i in range(2)])

table.extend([[f"true shower momentum {i}", true_start_shower_momentum[i]] for i in range(2)])
table.extend([[f"reco shower momentum {i}", reco_start_shower_momentum[i]] for i in range(2)])
table.extend([[f"reco PFO momentum {i}", reco_PFO_momentum[i]] for i in range(2)])

merged_momentum = [vector.vector(0, 0, 0)]*2
for j in range(2):
    for i in range(ak.count(reco_PFO_momentum[j].x)):
        merged_momentum[j] = vector.add(merged_momentum[j], reco_PFO_momentum[j][i])
    merged_momentum[j] = vector.add(merged_momentum[j], reco_start_shower_momentum[j])

table.extend([[f"merged momentum {i}:", merged_momentum[i]] for i in range(2)])

print(tabulate(table, tablefmt="fancy_grid"))


number of events: 87
number of signal PFOs: 2
╒════════════════════════╤══════════════════════════════════════════════════════════════════════╕
│ start shower energy 0  │ [141]                                                                │
├────────────────────────┼──────────────────────────────────────────────────────────────────────┤
│ start shower energy 1  │ [39.1]                                                               │
├────────────────────────┼──────────────────────────────────────────────────────────────────────┤
│ true shower energy 0   │ [0.0801]                                                             │
├────────────────────────┼──────────────────────────────────────────────────────────────────────┤
│ true shower energy 1   │ [0.144]                                                              │
├────────────────────────┼──────────────────────────────────────────────────────────────────────┤
│ signal PFO energy 0    │ [3.31, 2.2]                                  

In [219]:
import prod4a_evd
display = prod4a_evd.EventDisplay(events.eventNum[eventNum], events.run[eventNum], events.subRun[eventNum])
display.xy.set_facecolor("whitesmoke")
display.xz.set_facecolor("whitesmoke")

#* now Plot start showers:
points = events.recoParticles.spacePoints[start_showers_merged][eventNum]
startPoints = events.recoParticles.startPos[start_showers_merged][eventNum]
directions = events.recoParticles.direction[start_showers_merged][eventNum]
pdgs = events.trueParticlesBT.pdg[start_showers_merged][eventNum]
display.PlotPFO(points[0], marker = "x", colour = "green", startPoint = startPoints[0], direction = directions[0], pdg=pdgs[0])
display.PlotPFO(points[1], marker = "x", colour = "blue", startPoint = startPoints[1], direction = directions[1], pdg=pdgs[1])


points = events.recoParticles.spacePoints[to_merge][eventNum]
startPoints = events.recoParticles.startPos[to_merge][eventNum]
directions = events.recoParticles.direction[to_merge][eventNum]
pdgs = events.trueParticlesBT.pdg[to_merge][eventNum]
beam_mask = np.logical_not(events.recoParticles.number == events.recoParticles.beam_number)[to_merge][q.null]
#* Plot Signal PFOs
if showSignal:
    prod4a_evd.PlotSignalPFO(display, eventNum, signal, points, startPoints, directions, None, 0, -1, False)
    prod4a_evd.PlotSignalPFO(display, eventNum, signal, points, startPoints, directions, None, 1, -1, False)

#* Plot background PFOs
if showBackground: prod4a_evd.PlotBackgroundPFO(display, eventNum, background, beam_mask, points, startPoints, directions, pdg=None, i = -1, plotIP = False)

#* Plot BeamParticle:
beam_mask = events.recoParticles.number == events.recoParticles.beam_number
points = events.recoParticles.spacePoints[beam_mask][eventNum]
pdg = events.trueParticlesBT.pdg[beam_mask][eventNum]
display.PlotPFO(points, marker="o", colour="black", startPoint = events.recoParticles.beamVertex[eventNum], pdg=pdg)

#* Plot beam vertex
display.PlotPoint(events.recoParticles.beamVertex[eventNum], marker="x", colour="red", pointSize=100)

custom_lines = [matplotlib.lines.Line2D([0], [0], color="black", lw=2),
                matplotlib.lines.Line2D([0], [0], color="green", lw=2),
                matplotlib.lines.Line2D([0], [0], color="lime", lw=2),
                matplotlib.lines.Line2D([0], [0], color="blue", lw=2),
                matplotlib.lines.Line2D([0], [0], color="cyan", lw=2),
                matplotlib.lines.Line2D([0], [0], color="orange", lw=2),
                matplotlib.lines.Line2D([0], [0], marker="x", color="red", markersize=15, lw=0),
                ]

display.ax3D.legend(custom_lines, ["beam particle", "start shower 1", "signal 1", "start shower 2", "signal 2", "background", "decay vertex"])
display.xy.legend(custom_lines, ["beam particle", "start shower 1", "signal 1", "start shower 2", "signal 2", "background", "decay vertex"])
display.xy.grid()
display.xz.grid()
plt.tight_layout()
display.DetectorBounds()
plt.show()

Number of signal PFOs for start shower 0: 1
Number of signal PFOs for start shower 1: 0


/home/sb16165/anaconda3/envs/python3_6_8/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
